In [1]:
import findspark
findspark.init('C:\spark-2.3.1-bin-hadoop2.7')

import pyspark
sc = pyspark.SparkContext(appName="app2")


In [2]:
from pyspark.mllib.util import MLUtils
libsvmData = MLUtils.loadLibSVMFile(sc, '../datasets/wine.scale')
from pyspark.mllib.tree import DecisionTree

In [3]:
libsvmData

PythonRDD[4] at RDD at PythonRDD.scala:49

In [4]:
libsvmData.take(5)

[LabeledPoint(1.0, (13,[0,1,2,3,4,5,6,7,8,9,10,11,12],[0.68421,-0.616601,0.144385,-0.484536,0.23913,0.255172,0.147679,-0.433962,0.18612,-0.255973,-0.089431,0.941392,0.122682])),
 LabeledPoint(1.0, (13,[0,1,2,3,4,5,6,7,8,9,10,11,12],[0.142105,-0.588933,-0.165775,-0.938144,-0.347826,0.151724,0.0210971,-0.509434,-0.451104,-0.47099,-0.0731708,0.56044,0.101284])),
 LabeledPoint(1.0, (13,[0,1,2,3,4,5,6,7,8,9,10,11,12],[0.121053,-0.359684,0.40107,-0.175258,-0.326087,0.255172,0.223629,-0.358491,0.514196,-0.249147,-0.105691,0.391941,0.293866])),
 LabeledPoint(1.0, (13,[0,1,2,3,4,5,6,7,8,9,10,11,12],[0.757895,-0.521739,0.219251,-0.360825,-0.0652174,0.97931,0.329114,-0.584906,0.116719,0.112628,-0.382114,0.59707,0.714693])),
 LabeledPoint(1.0, (13,[0,1,2,3,4,5,6,7,8,9,10,11,12],[0.163158,-0.268775,0.614973,0.0721649,0.0434783,0.255172,-0.00843878,-0.018868,-0.11041,-0.481229,-0.089431,0.216117,-0.348074]))]

In [5]:
(trainingData, testData) = libsvmData.randomSplit([0.7,0.3])
libsvmModel = DecisionTree.trainClassifier( trainingData,
                                          numClasses = 4,
                                          categoricalFeaturesInfo = {},
                                          impurity = 'gini',
                                          maxDepth = 5,
                                          maxBins = 32)

predictions = libsvmModel.predict(testData.map(lambda x:x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
labelsAndPredictions.take(5)



[(1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]

In [6]:
from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(labelsAndPredictions)
metrics.accuracy

0.9183673469387755

In [7]:
metrics.confusionMatrix().toArray()

array([[15.,  3.,  0.],
       [ 1., 15.,  0.],
       [ 0.,  0., 15.]])

In [9]:
print(libsvmModel.toDebugString())

DecisionTreeModel classifier of depth 4 with 15 nodes
  If (feature 9 <= -0.5614334999999999)
   If (feature 0 <= 0.057894799999999996)
    Predict: 2.0
   Else (feature 0 > 0.057894799999999996)
    If (feature 1 <= -0.6343875)
     Predict: 2.0
    Else (feature 1 > -0.6343875)
     Predict: 1.0
  Else (feature 9 > -0.5614334999999999)
   If (feature 6 <= -0.478903)
    If (feature 3 <= -0.3505155)
     Predict: 2.0
    Else (feature 3 > -0.3505155)
     Predict: 3.0
   Else (feature 6 > -0.478903)
    If (feature 12 <= -0.375178)
     If (feature 0 <= 0.057894799999999996)
      Predict: 2.0
     Else (feature 0 > 0.057894799999999996)
      Predict: 1.0
    Else (feature 12 > -0.375178)
     Predict: 1.0

